<a href="https://colab.research.google.com/github/sahniaditya007/Crypto-Bot/blob/harshit/p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install numpy
!pip install pandas
!pip installbinance
!pip install scikit-learn
!pip installtensorflow
!pip install python-binance

ERROR: unknown command "installbinance" - maybe you meant "install"
ERROR: unknown command "installtensorflow"


In [27]:
import numpy as np
import pandas as pd
import time
from binance.client import Client
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense

In [28]:
# Binance Testnet API Credentials (Replace with your own keys from Testnet)
from binance.client import Client

API_KEY = "SqQl1AFmjzYiIKvnt1QQoSb8MUn5ussbH1KgkRZiAuHbxCR0SUfeM4SkOHOywU9G"
API_SECRET = "kIEa5qbDAf1LVHO84orgNPR5ADqx6pJwcPNgFdlvxKIv9r2E7Ibpuhc1M1DTqlSn"
client = Client(API_KEY, API_SECRET, tld='com', testnet=True)

In [51]:
# Function to fetch historical OHLCV data
def get_historical_data(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_1MINUTE, lookback='365 days ago UTC'):
    try:
        klines = client.get_historical_klines(symbol, interval, lookback)
        df = pd.DataFrame(klines, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time',
                                           'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
                                           'Taker buy quote asset volume', 'Ignore'])
        df = df[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
        return df
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        return None

data = get_historical_data()
if data is None:
    raise RuntimeError("Failed to fetch historical data!")

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [52]:
# Function to create time-series sequences
def create_sequences(data, time_steps=60):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps, 3])  # Predicting 'Close' price
    return np.array(X), np.array(y)

TIME_STEPS = 60
X, y = create_sequences(data_scaled, TIME_STEPS)
X_train, X_test, y_train, y_test = X[:int(0.8*len(X))], X[int(0.8*len(X)):], y[:int(0.8*len(y))], y[int(0.8*len(y)):]


In [53]:
# Build Advanced LSTM Model
def build_model():
    inputs = Input(shape=(TIME_STEPS, 5))
    x = Bidirectional(LSTM(128, return_sequences=True))(inputs)
    x = Dropout(0.2)(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    attention = Attention()([x, x])
    x = Bidirectional(LSTM(64, return_sequences=False))(attention)
    x = Dropout(0.2)(x)
    output = Dense(1, activation='linear')(x)
    model = tf.keras.Model(inputs, output)
    model.compile(optimizer=Adam(learning_rate=0.001, decay=1e-6), loss='mse')
    return model

In [55]:
# Train the model
def train_model(model, X_train, y_train, X_test, y_test, epochs=100, batch_size=32):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))
    return model

model = train_model(model, X_train, y_train, X_test, y_test)

Epoch 1/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 9.6901e-05 - val_loss: 7.0803e-07
Epoch 2/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 1.4011e-05 - val_loss: 7.4936e-07
Epoch 3/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 1.0943e-05 - val_loss: 8.5392e-07
Epoch 4/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 1.0785e-04 - val_loss: 7.4174e-07
Epoch 5/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 3.2855e-05 - val_loss: 8.0786e-07
Epoch 6/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 6.5699e-05 - val_loss: 7.0540e-07
Epoch 7/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 2.3296e-05 - val_loss: 7.5938e-07
Epoch 8/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: 9.3447e-06 - val_loss: 1.0613e-06
Epoch 9/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 1.1390e-05 - val_loss: 1.6394e-06
Epoch 10/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 1.1019e-04 - val_loss: 6.7825e-07
Epoch 11/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 1

In [56]:
def predict_next_60_minutes():
    try:
        latest_data = get_historical_data().values[-TIME_STEPS:]
        latest_scaled = scaler.transform(latest_data)
        latest_scaled = np.expand_dims(latest_scaled, axis=0)
        prediction = model.predict(latest_scaled)
        predicted_prices = scaler.inverse_transform(np.hstack((np.zeros((1, 4)), prediction.reshape(-1, 1))))[:, 3]
        return predicted_prices
    except Exception as e:
        print(f"Error predicting next 60 minutes: {e}")
        return None

In [57]:
# Function for simulated semi-automated trade execution
def execute_trade(symbol='BTCUSDT', trade_size=0.001, stop_loss_pct=0.02, take_profit_pct=0.05):
    predicted_price = predict_next_close()
    if predicted_price is None:
        print("Skipping trade due to prediction failure.")
        return

    try:
        current_price = float(client.get_symbol_ticker(symbol=symbol)['price'])
        stop_loss = current_price * (1 - stop_loss_pct)
        take_profit = current_price * (1 + take_profit_pct)

        print(f"Predicted Price: {predicted_price}, Current Price: {current_price}")
        print(f"Stop-Loss Price: {stop_loss}, Take-Profit Price: {take_profit}")

        decision = input("Enter 'BUY' or 'SELL' or 'SKIP': ").strip().upper()
        if decision == 'BUY':
            order = client.create_test_order(symbol=symbol, side='BUY', type='MARKET', quantity=trade_size)
            print("Simulated Buy Order Executed:", order)
        elif decision == 'SELL':
            order = client.create_test_order(symbol=symbol, side='SELL', type='MARKET', quantity=trade_size)
            print("Simulated Sell Order Executed:", order)
        else:
            print("No trade executed.")
    except Exception as e:
        print(f"Error executing trade: {e}")

In [58]:
# Running real-time prediction & trading loop (Simulated)
try:
    while True:
        execute_trade()
        time.sleep(1)  # Wait 1 minute before next execution
except KeyboardInterrupt:
    print("\nTrading bot stopped.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Predicted Price: 87617.62576559841, Current Price: 87639.2
Stop-Loss Price: 85886.416, Take-Profit Price: 92021.16
Enter 'BUY' or 'SELL' or 'SKIP': skip
No trade executed.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Predicted Price: 87579.09426769793, Current Price: 87655.26
Stop-Loss Price: 85902.15479999999, Take-Profit Price: 92038.023
Enter 'BUY' or 'SELL' or 'SKIP': skip
No trade executed.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


...

Predicted Price: 87614.32665177167, Current Price: 87675.38
Stop-Loss Price: 85921.87240000001, Take-Profit Price: 92059.149
Enter 'BUY' or 'SELL' or 'SKIP': SKIP
No trade executed.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Predicted Price: 87606.56297629893, Current Price: 87678.74
Stop-Loss Price: 85925.1652, Take-Profit Price: 92062.67700000001
Enter 'BUY' or 'SELL' or 'SKIP': SKIP
No trade executed.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Predicted Price: 87608.16770829618, Current Price: 87676.93
Stop-Loss Price: 85923.3914, Take-Profit Price: 92060.77649999999

Trading bot stopped.
